# Segmentation

## Importing dependencies

In [ ]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from skimage import morphology, io, color, exposure, img_as_float, transform
from matplotlib import pyplot as plt
import openslide as op
from PIL import Image
from util import IoU, Dice, masked, remove_small_regions, normalize
from scipy import misc

import time

## Configuration

In [ ]:
# Paths
filename = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.svs'
maskname = '/root/workspace/data/preExtraction/15_02/IFTA_15_02.png'

model_name = './model.099.hdf5'

# Do not touch
level=1
patchSize = 256
colorChannels = 3
stride = patchSize/2

patchShape = (patchSize, patchSize,colorChannels) # Shape of the image (patch)
patchMaskShape = (patchSize, patchSize)   # Shape of the mask (patch)

## Image loading

### Loading base picture

In [ ]:
im = op.OpenSlide(filename)
imload = np.asfarray(im.read_region((0,0), 1, im.level_dimensions[1]),dtype=np.float32)[:,:,0:colorChannels]/255

#normalization
imload-=imload.mean()
imload/=imload.std()

### Loading mask

In [ ]:
maskload = Image.open(maskname)
maskload = np.expand_dims(np.array(maskload.resize(im.level_dimensions[level]),dtype=np.float32),-1)

### Allocating final array

In [ ]:
#output = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0]),dtype=np.float32)

In [ ]:
outputThreshold = np.zeros((im.level_dimensions[level][1],im.level_dimensions[level][0], colorChannels),dtype=np.int8)

## Loading model

In [ ]:
# Load model
UNet = load_model(model_name)

## Execution

### Initialization  

In [ ]:
test_gen = ImageDataGenerator(rescale=1.)

### Prediction

In [ ]:
t0 = time.clock()

# Prediction
for posX in range(0,im.level_dimensions[level][0]-256,256):
    for posY in range(0,im.level_dimensions[level][1]-256,256):
        
        xx = np.expand_dims(imload[posY:posY+256,posX:posX+256,:],0)
        yy = maskload[posY:posY+256,posX:posX+256,:]

        
        #img = exposure.rescale_intensity(np.squeeze(xx), out_range=(0,1))
        pred = UNet.predict(xx)[..., 0].reshape(patchShape[:2])
        mask = yy[..., 0].reshape(patchShape[:2])
        gt = mask > 0.5
        pr = pred > 0.5
        pr = remove_small_regions(pr, 0.02 * np.prod(patchShape))
        #io.imsave('{}'.format(df.iloc[i].path), masked(img, gt, pr, 1))
        #gts.append(gt)
        #prs.append(pr)
        #ious[i] = IoU(gt, pr)
        #dices[i] = Dice(gt, pr)
        #print(i, ious[i], dices[i])
        
        #outputPatches[i,:,:,:] = np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8)))
        outputPart = np.dstack((pr.astype(np.int8), gt.astype(np.int8), pr.astype(np.int8)))
        #if i < 4:
        #    plt.subplot(4, 4, 4*i+1)
        #    plt.title('Processed ' + str(i))
        #    plt.axis('off')
        #    plt.imshow(img, cmap='gray')
        #    plt.subplot(4, 4, 4 * i + 2)
        #    plt.title('IoU = {:.4f}'.format(ious[i]))
        #    plt.axis('off')
        #    plt.imshow(masked(img, gt, pr, 1))
        #    plt.subplot(4, 4, 4*i+3)
        #    plt.title('Prediction')
        #    plt.axis('off')
        #    plt.imshow(pred, cmap='jet')
        #    plt.subplot(4, 4, 4*i+4)
        #    plt.title('Difference')
        #    plt.axis('off')
        #    plt.imshow(outputPart)

        #Add the prediction patch to the output picture


        #if i!=0 and i % modulo == 0:
        #    iterX=0
        #    iterY+=1


        part=outputThreshold[posY:posY+256,posX:posX+256,:]
        part[:,:,:]=outputPart[0:part.shape[0],0:part.shape[1],:]

        #part=output[posY:posY+256,posX:posX+256]
        #part[:,:]=pred[0:part.shape[0],0:part.shape[1]]

t = time.clock()
print("Execution duration : ", t-t0)

In [ ]:
plt.figure(figsize = (20,20)) # create a 5 x 5 figure *
plt.subplot(211)
plt.imshow(outputThreshold,interpolation='none', cmap="jet")
plt.show()

In [ ]:
misc.imsave("prediction_15-02_voley.png", outputThreshold)